In [1]:
import face_recognition

# Load face images and encode them
known_face_encodings = []
known_face_names = []

for person_name in ['modi', 'trump', 'subha']:
    images = [f'persons/{person_name}/{i}.jpeg' for i in range(1, 6)]  # Assuming images are named 1.jpeg, 2.jpeg, ...
    for image_path in images:
        image = face_recognition.load_image_file(image_path)
        encoding = face_recognition.face_encodings(image)[0]
        known_face_encodings.append(encoding)
        known_face_names.append(person_name)

In [2]:
import cv2

In [3]:


video_capture = cv2.VideoCapture(0)

while True:
    ret, frame = video_capture.read()
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"

        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]

        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)

    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/subha/Desktop/Projects/bdas_system/bdas/lib/python3.11/site-packages/cv2/qt/plugins"


In [6]:
image = cv2.imread('persons/modi/1.jpeg')
# cv2.imwrite('modi1.jpeg', image)
print(image)

None


In [10]:
import hashlib

def calculate_md5(filename):
    hash_md5 = hashlib.md5()
    with open(filename, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

# Replace 'example.jpg' with the actual path to your image file
image_hash = calculate_md5('persons/modi/1.jpeg')

print("MD5 hash of the image:", image_hash)

MD5 hash of the image: 68ae04815297730ae9cb106f62e58fa8


In [13]:
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from base64 import urlsafe_b64encode, urlsafe_b64decode

def generate_key_from_password(password, salt):
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        iterations=100000,
        salt=salt,
        length=32,
        backend=default_backend()
    )
    key = kdf.derive(password.encode('utf-8'))
    return key

def pad_data(data):
    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(data) + padder.finalize()
    return padded_data

def unpad_data(data):
    unpadder = padding.PKCS7(128).unpadder()
    unpadded_data = unpadder.update(data) + unpadder.finalize()
    return unpadded_data

import os

def generate_random_iv():
    return os.urandom(16)

def encrypt_image(input_file, output_file, password):
    salt = b'salt_'
    key = generate_key_from_password(password, salt)
    iv = generate_random_iv()
    
    with open(input_file, 'rb') as f:
        image_data = f.read()
    
    padded_data = pad_data(image_data)
    
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    cipher_text = encryptor.update(padded_data) + encryptor.finalize()
    
    with open(output_file, 'wb') as f:
        f.write(salt + iv + cipher_text)

def decrypt_image(input_file, output_file, password):
    with open(input_file, 'rb') as f:
        data = f.read()
    
    salt = data[:5]
    iv = data[5:21]  # IV is now 16 bytes long
    cipher_text = data[21:]
    
    key = generate_key_from_password(password, salt)
    
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    padded_data = decryptor.update(cipher_text) + decryptor.finalize()
    
    image_data = unpad_data(padded_data)
    
    with open(output_file, 'wb') as f:
        f.write(image_data)

# Usage
encrypt_image('persons/modi/1.jpeg', 'encrypted_image.jpg', 'password123')
decrypt_image('encrypted_image.jpg', 'decrypted_image.jpg', 'password123')